In [70]:
import numpy as np
import pandas as pd
import cv2
import matplotlib.pyplot as plt
from PIL import Image
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
import os


In [71]:
!pip install cairosvg
!pip install svgpathtools cairosvg


In [72]:
import cairosvg
import svgpathtools

In [73]:
def load_data(csv_path):
    data = pd.read_csv(csv_path, header=None, names=["PathID", "PointID", "X", "Y"])
    return data

def clean_data(data):
    data['PathID'] = data['PathID'].astype(int)
    data['PointID'] = data['PointID'].astype(int)
    cleaned_data = data.drop_duplicates(subset=['PathID', 'PointID', 'X', 'Y']).reset_index(drop=True)
    return cleaned_data

def visualize_data(data):
    plt.scatter(data['X'], data['Y'], c='blue', marker='o')
    plt.title('Scatter Plot of Raw Data')
    plt.xlabel('X')
    plt.ylabel('Y')
    plt.show()

def segment_paths(data, threshold=10):
    data['NewPathID'] = 0
    last_index = 0
    for i in range(1, len(data)):
        dist = np.linalg.norm(data.iloc[i][['X', 'Y']].values - data.iloc[i-1][['X', 'Y']].values)
        if dist > threshold:
            last_index += 1
        data.at[i, 'NewPathID'] = last_index
    return data

def normalize_data(data):
    data['X'] = (data['X'] - data['X'].min()) / (data['X'].max() - data['X'].min())
    data['Y'] = (data['Y'] - data['Y'].min()) / (data['Y'].max() - data['Y'].min())
    return data

def augment_data(data):
    theta = np.radians(30)
    c, s = np.cos(theta), np.sin(theta)
    data['X_rot'] = data['X'] * c - data['Y'] * s
    data['Y_rot'] = data['X'] * s + data['Y'] * c
    scale_factor = 1.2
    data['X_scale'] = data['X'] * scale_factor
    data['Y_scale'] = data['Y'] * scale_factor
    translate_x, translate_y = 0.1, 0.1
    data['X_trans'] = data['X'] + translate_x
    data['Y_trans'] = data['Y'] + translate_y
    return data


In [74]:
def bezier_curve(P0, P1, P2, P3, t):
    return (1-t)**3 * P0 + 3*(1-t)**2 * t * P1 + 3*(1-t) * t**2 * P2 + t**3 * P3

def fit_bezier_curve(data, n_segments=10):
    segments = np.array_split(data[['X', 'Y']].values, n_segments)
    bezier_paths = []
    for segment in segments:
        P0 = segment[0]
        P3 = segment[-1]
        P1 = segment[len(segment) // 3]
        P2 = segment[2 * len(segment) // 3]
        t_values = np.linspace(0, 1, 100)
        bezier_points = np.array([bezier_curve(P0, P1, P2, P3, t) for t in t_values])
        bezier_paths.append(bezier_points)
    return bezier_paths

def polyline_to_image(polyline, image_size=(64, 64), line_thickness=2):
    image = np.ones(image_size, dtype=np.uint8) * 255
    polyline = np.array(polyline)
    polyline[:, 0] = np.interp(polyline[:, 0], (polyline[:, 0].min(), polyline[:, 0].max()), (0, image_size[0] - 1))
    polyline[:, 1] = np.interp(polyline[:, 1], (polyline[:, 1].min(), polyline[:, 1].max()), (0, image_size[1] - 1))
    polyline = polyline.astype(int)
    cv2.polylines(image, [polyline], isClosed=False, color=(0,), thickness=line_thickness)
    return image

def save_polyline_as_image(points, path, file_name):
    plt.figure(figsize=(64 / 100, 64 / 100), dpi=100)
    plt.plot(points[:, 0], points[:, 1], marker='o')
    plt.xlim(0, 1)
    plt.ylim(0, 1)
    plt.gca().invert_yaxis()
    plt.axis('off')
    os.makedirs(path, exist_ok=True)
    plt.savefig(os.path.join(path, file_name), bbox_inches='tight', pad_inches=0, cmap='gray')
    plt.close()

def generate_images_from_csv(data, output_path):
    for path_id in data['PathID'].unique():
        subset = data[data['PathID'] == path_id]
        polyline = subset[['X', 'Y']].values
        image = polyline_to_image(polyline)
        Image.fromarray(image).save(os.path.join(output_path, f'path_{path_id}.png'))


In [75]:
def create_directory(path):
    os.makedirs(path, exist_ok=True)

def process_svg(svg_path, output_path):
    """
    Convert an SVG file to PNG and save it in the specified directory.

    Args:
    - svg_path (str): Path to the SVG file.
    - output_path (str): Directory to save the converted PNG file.
    """
    # Create the output directory if it doesn't exist
    create_directory(output_path)

    # Define the path for the PNG file
    png_path = os.path.join(output_path, 'converted_image.png')

    # Convert SVG to PNG
    cairosvg.svg2png(url=svg_path, write_to=png_path)

    print(f"Converted SVG file saved to: {png_path}")

def svg_to_points(svg_path):
    """
    Extract points from an SVG path file and return them.

    Args:
    - svg_path (str): Path to the SVG file.

    Returns:
    - points (list): List of (x, y) tuples.
    """
    paths, attributes = svgpathtools.svg2paths(svg_path)
    points = []

    for path in paths:
        for segment in path:
            if isinstance(segment, svgpathtools.Line):
                points.append((segment.start.real, segment.start.imag))
                points.append((segment.end.real, segment.end.imag))
            elif isinstance(segment, svgpathtools.CubicBezier):
                t_values = np.linspace(0, 1, 10)
                for t in t_values:
                    point = segment.point(t)
                    points.append((point.real, point.imag))

    return points

def plot_points(points, output_file):
    """
    Plot points and save the plot as an image.

    Args:
    - points (list): List of (x, y) tuples.
    - output_file (str): Path to save the image.
    """
    x, y = zip(*points)

    plt.figure(figsize=(10, 10))
    plt.plot(x, y, 'o')
    plt.title('SVG Path Points')
    plt.xlabel('X')
    plt.ylabel('Y')
    plt.gca().invert_yaxis()
    plt.axis('equal')
    plt.savefig(output_file, bbox_inches='tight')
    plt.close()

In [84]:
csv_path1 = 'https://raw.githubusercontent.com/Harshsinghr/Curvetopia-Adobe-GenSolve-Round2/main/Data/problems/occlusion1.csv'
csv_path2 = 'https://raw.githubusercontent.com/Harshsinghr/Curvetopia-Adobe-GenSolve-Round2/main/Data/problems/occlusion2.csv'
output_dir1 = '/output_images/occlusion1'
output_dir2 = '/output_images/occlusion2'
create_directory(output_dir1)
create_directory(output_dir2)
svg_path1 = 'https://raw.githubusercontent.com/Harshsinghr/Curvetopia-Adobe-GenSolve-Round2/main/Data/problems/occlusion1.svg'
svg_path2 = 'https://raw.githubusercontent.com/Harshsinghr/Curvetopia-Adobe-GenSolve-Round2/main/Data/problems/occlusion2.svg'

process_csv(csv_path1, output_dir1)
process_csv(csv_path2, output_dir2)

svg_output_dir1 = '/output_images/occlusion1_svg'
svg_output_dir2 = '/output_images/occlusion2_svg'

# Convert SVG to PNG
process_svg(svg_path1, svg_output_dir1)
process_svg(svg_path2, svg_output_dir2)

# Extract points from SVG and plot
points1 = svg_to_points(svg_path1)
points2 = svg_to_points(svg_path2)

# Save plotted points as images
plot_points(points1, os.path.join(svg_output_dir1, 'points_plot.png'))
plot_points(points2, os.path.join(svg_output_dir2, 'points_plot.png'))



In [82]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model

# Define and compile the model
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(64, 64, 3))
x = base_model.output
x = Flatten()(x)
x = Dense(256, activation='relu')(x)
x = Dense(6, activation='softmax')(x)  # Number of classes

model = Model(inputs=base_model.input, outputs=x)

# Freeze the base model layers
for layer in base_model.layers:
    layer.trainable = False

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Set up data generators for training
# Set up data generators with improved augmentation
datagen = ImageDataGenerator(
    rescale=1.0/255.0,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)


In [83]:
image_dir = '/output_images'  # Update with your actual path
train_generator = datagen.flow_from_directory(
    '/output_images',
    target_size=(64, 64),
    color_mode='rgb',  # Changed to RGB for pre-trained model
    class_mode='categorical',
    batch_size=16
)

# Define callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
lr_reduction = ReduceLROnPlateau(monitor='val_loss', patience=3, verbose=1, factor=0.5)

# Train the model
history = model.fit(train_generator, epochs=20, callbacks=[early_stopping, lr_reduction])

print("Model training complete")


Found 13 images belonging to 6 classes.
Epoch 1/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 8s 8s/step - accuracy: 0.1538 - loss: 1.8759 - learning_rate: 0.0010
Epoch 2/20


/usr/local/lib/python3.10/dist-packages/keras/src/callbacks/early_stopping.py:155: UserWarning: Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: accuracy,loss
  current = self.get_monitor_value(logs)
/usr/local/lib/python3.10/dist-packages/keras/src/callbacks/callback_list.py:96: UserWarning: Learning rate reduction is conditioned on metric `val_loss` which is not available. Available metrics are: accuracy,loss,learning_rate.
  callback.on_epoch_end(epoch, logs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 301ms/step - accuracy: 0.3846 - loss: 1.6408 - learning_rate: 0.0010
Epoch 3/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 270ms/step - accuracy: 0.6154 - loss: 1.3138 - learning_rate: 0.0010
Epoch 4/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 283ms/step - accuracy: 0.6923 - loss: 1.2224 - learning_rate: 0.0010
Epoch 5/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 332ms/step - accuracy: 0.7692 - loss: 1.0516 - learning_rate: 0.0010
Epoch 6/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 268ms/step - accuracy: 0.7692 - loss: 0.8438 - learning_rate: 0.0010
Epoch 7/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 268ms/step - accuracy: 0.6923 - loss: 0.7488 - learning_rate: 0.0010
Epoch 8/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 266ms/step - accuracy: 0.6923 - loss: 0.7463 - learning_rate: 0.0010
Epoch 9/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 268ms/step - accuracy: 0.6923 - loss: 0.6996 - learning_rate: 0.0010
Epoch 10/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 263ms/step - accuracy: 0.6923 - loss: 0.9380 - learning_rate: 0.0010
Epoch 11/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0